# Topic diversity exploration

In [4]:
%pip install datasets pandas scikit-learn umap-learn

Note: you may need to restart the kernel to use updated packages.


## K-means clustering

In [40]:
import pandas as pd
import numpy as np
df = pd.read_parquet("datasets/posts-title-url-bge.parquet")
df['embeddings'] = df['embeddings'].apply(lambda x: np.array(x, dtype='float32'))

Let's, totally ad-hoc, start us off with some random hyperparameters because I am tired and want to go to bed.

Training on 10k random titles and 10 clusters to just validate the pipeline, I will test and optimize tomorrow.

In [41]:
np.random.seed(42)

df_train_sample = df.sample(n=25_000, random_state=42)

In [42]:
train_emb_array = np.array(df_train_sample["embeddings"].tolist())

In [43]:
import umap.umap_ as umap

# Reduce dimensionality of train_emb_array using UMAP to 3 dimensions
reducer = umap.UMAP(n_components=32, n_jobs=-1)
train_emb_reduced = reducer.fit_transform(train_emb_array)

In [44]:
from sklearn.cluster import MiniBatchKMeans

silhouette_avgs = {}
db_scores = {}

# Powers of 2 of k values to try
k_values = [2**i for i in range(1, 10)]

from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score

for n_clusters in k_values:
    # Initialize MiniBatchKMeans
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, n_init="auto")
    
    # Fit the model
    kmeans = kmeans.fit(train_emb_reduced)
    
    # Get cluster labels
    labels_sample = kmeans.labels_
    
    # Compute Davies-Bouldin score
    db_score = davies_bouldin_score(train_emb_reduced, labels_sample)

    """
    print("Silhouette scores per cluster:")
    for label, score in silhouette_dict.items():
     # Get score for each sample
    silhouette_vals = silhouette_samples(train_emb_reduced, labels_sample, metric='cosine')

    # Save silhouette score for each cluster
    silhouette_dict = {}
    for label in np.unique(labels_sample):
        label_silhouette_vals = silhouette_vals[labels_sample == label]
        silhouette_dict[label] = label_silhouette_vals.mean()

       print(f"Cluster {label}: {score}")
    """
    # Compute silhouette score
    silhouette_avg = silhouette_score(train_emb_reduced, labels_sample, metric='cosine', n_jobs=-1)

    print(f"For n_clusters = {n_clusters}, the average silhouette_score is : {silhouette_avg}")
    print(f"The average Davies-Bouldin score is: {db_score }")
    
    # Store silhouette_avg in dictionary
    silhouette_avgs[n_clusters] = silhouette_avg
    db_scores[n_clusters] = db_score
    

For n_clusters = 2, the average silhouette_score is : 0.5347391366958618
The average Davies-Bouldin score is: 1.156073779955633
For n_clusters = 4, the average silhouette_score is : 0.4585530757904053
The average Davies-Bouldin score is: 1.2622611339759329
For n_clusters = 8, the average silhouette_score is : 0.39565014839172363
The average Davies-Bouldin score is: 1.2398232937700149
For n_clusters = 16, the average silhouette_score is : 0.42537006735801697
The average Davies-Bouldin score is: 1.1577835377205634
For n_clusters = 32, the average silhouette_score is : 0.43907004594802856
The average Davies-Bouldin score is: 1.1424023424213279
For n_clusters = 64, the average silhouette_score is : 0.4447826147079468
The average Davies-Bouldin score is: 1.1824426705742355
For n_clusters = 128, the average silhouette_score is : 0.4404447376728058
The average Davies-Bouldin score is: 1.1568903313518404
For n_clusters = 256, the average silhouette_score is : 0.4161699116230011
The average Dav

In [7]:
%pip install matplotlib

  Obtaining dependency information for pillow>=6.2.0 from https://files.pythonhosted.org/packages/7a/07/e896b096a77375e78e02ce222ae4fd6014928cd76c691d312060a1645dfa/Pillow-10.0.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata
Using cached Pillow-10.0.1-cp310-cp310-manylinux_2_28_x86_64.whl (3.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rembg 2.0.25 requires click==8.1.3, which is not installed.
rembg 2.0.25 requires fastapi==0.80.0, which is not installed.
diffusers 0.8.0.dev0 requires Pillow<10.0, but you have pillow 10.0.1 which is incompatible.
rembg 2.0.25 requires numpy==1.21.6, but you have numpy 1.25.2 which is incompatible.
rembg 2.0.25 requires pillow==9.2.0, but you have pillow 10.0.1 which is incompatible.
rembg 2.0.25 requires scipy==1.7.3, but you have scipy 1.11.2 which is incompatible.
rembg 2.0.25 requires tqdm==4.64.0, but you have tq

In [8]:
import matplotlib.pyplot as plt

# Plotting the silhouette scores
plt.figure(figsize=(16, 8))
plt.plot(list(silhouette_avgs.keys()), list(silhouette_avgs.values()))
plt.title("Silhouette Scores vs Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")
plt.show()

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [46]:
n_clusters=512

In [47]:
kmeans = MiniBatchKMeans(n_clusters=n_clusters, n_init="auto")

kmeans = kmeans.fit(train_emb_reduced)

Optional: Remove duplicate columns. Idempotent if not

In [10]:
# Remove duplicates
# Drop columns ending with '_y'
columns_to_drop = [col for col in df.columns if col.endswith('_y')]
df.drop(columns=columns_to_drop, inplace=True)

# Rename columns ending with '_x' to remove the suffix
columns_to_rename = {col: col.rstrip('_x') for col in df.columns if col.endswith('_x')}
df.rename(columns=columns_to_rename, inplace=True)


In [45]:
import numpy as np

embeddings_matrix = reducer.transform(np.stack(df["embeddings"].to_numpy()).astype(np.float32))

In [49]:
df["title_cluster"] = kmeans.predict(embeddings_matrix)
df["title_cluster"].value_counts()

title_cluster
39     4243
320    3827
30     3441
327    3031
57     2988
       ... 
261     137
58      124
180     114
183      99
376      40
Name: count, Length: 512, dtype: int64

In [50]:
# Function to print 5 representative titles for each cluster
def print_representative_titles(df, cluster_col='title_cluster', title_col='title'):
    unique_clusters = df[cluster_col].unique()
    
    for cluster in unique_clusters:
        representative_titles = df[df[cluster_col] == cluster][title_col].sample(7, replace=True).tolist()
        print(f"Cluster {cluster} Representative Titles:")
        for title in representative_titles:
            print(f"  - {title}")

print_representative_titles(df)

Cluster 130 Representative Titles:
  - Apple’s use of Swift and SwiftUI in iOS 15
  - Buffereditor – Code and Text Editor for iPhone, iPad and iPad Pro
  - Homebridge emulates the iOS Homekit API
  - Exploring Android Development on iPad
  - The Safari bug that never was (2022)
  - Show HN: I'm 14, please check out my iOS game
  - How to create HTML5 web apps with Xcode4
Cluster 127 Representative Titles:
  - Ask HN: Who is hiring? (August 2022)
  - Ask HN: Why don't we have 3 day weekends yet?
  - Ask HN: Who is hiring? (November 2019)
  - Ask HN: YC applications are up. Did you submit?
  - Ask HN: Is it wise to accept an intern position at Meta?
  - Ask HN: Who's sick of remote work?
  - Tell candidates what to expect from your job interviews
Cluster 191 Representative Titles:
  - The Pmarca Guide to Career Planning, Part 1
  - The Udemy Pyramid Scheme – Why I stopped Teaching at Udemy
  - An Amaz-ing Resume
  - How to Ask Useful Questions
  - Tech Education Con
  - Good Product Mana

## Topic modeling

Can we make these clusters more legible? Copy-pasting from Maarten Grootendorst's [c-TF-IDF](https://www.maartengrootendorst.com/blog/ctfidf/) (though not the whole BERTopic skeleton yet):

In [26]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer


class CTFIDFVectorizer(TfidfTransformer):
    def __init__(self, *args, **kwargs):
        super(CTFIDFVectorizer, self).__init__(*args, **kwargs)

    def fit(self, X: sp.csr_matrix, n_samples: int):
        """
        n_samples: Number of unjoined documents.

        Learn the idf vector (global term weights) 
        """
        _, n_features = X.shape
        df = np.squeeze(np.asarray(X.sum(axis=0)))
        idf = np.log(n_samples / df)
        self._idf_diag = sp.diags(idf, offsets=0,
                                  shape=(n_features, n_features),
                                  format='csr',
                                  dtype=np.float64)
        return self

    def transform(self, X: sp.csr_matrix) -> sp.csr_matrix:
        """Transform a count-based matrix to c-TF-IDF """
        X = X * self._idf_diag
        X = normalize(X, axis=1, norm='l1', copy=False)
        return X

Let's apply this to the whole fucking set:

In [27]:
# Concat all docs per class in train set
docs_per_class_train = df.groupby('title_cluster', as_index=False).agg({'title': ' '.join})

# Create bag of words
count_vectorizer = CountVectorizer().fit(docs_per_class_train.title)
count = count_vectorizer.transform(docs_per_class_train.title)
words = count_vectorizer.get_feature_names_out()

# Create c-tfidf vectorizer
ctfidf = CTFIDFVectorizer().fit_transform(count, n_samples=len(docs_per_class_train))

OK, let's see some of the results:

In [28]:
ctfidf_arr = ctfidf.toarray()
words_per_cluster = [[words[index] for index in ctfidf_arr[cluster].argsort()[-10:]] for cluster in range(n_clusters)]

In [29]:
cluster_counts = df["title_cluster"].value_counts()
cluster_counts.sort_index(inplace=True)
cluster_df = pd.DataFrame({"cluster": cluster_counts.index, "count": cluster_counts.values, "top_words": [", ".join(x) for x in words_per_cluster]})
cluster_df.sort_values("count", ascending=False, inplace=True)
cluster_df.head(20)

,cluster,count,top_words
265,265,3486,"shakespeare, poetry, eliot, samuel, tolkien, p..."
9,9,3375,"teslas, ford, motors, ion, batteries, toyota, ..."
5,5,3218,"f8, metaverse, snapchat, delete, advertisers, ..."
209,209,3180,"mpaa, torrents, hbo, hulu, piracy, drm, dmca, ..."
26,26,3107,"skylake, processors, radeon, i9, threadripper,..."
197,197,3090,"spyware, foia, fisa, prism, patriot, revelatio..."
35,35,2994,"nixos, rhel, fedora, lts, arch, 04, centos, di..."
333,333,2913,"bootstrapping, founding, cofounders, solo, ent..."
422,422,2825,"rusty, rustc, rustconf, crate, async, crates, ..."
500,500,2663,"whistleblower, fb, gawker, influencers, teens,..."


In [177]:
cluster_df.to_csv("datasets/title-cluster-top-words.csv", index=False)

In [171]:
df["title_cluster"].value_counts()

title_cluster
55     4517
323    3758
326    2872
151    2856
163    2773
       ... 
411     141
188     137
249     136
290     117
179      76
Name: count, Length: 512, dtype: int64

In [65]:
df_silhouette_sample = df.sample(n=10_000, random_state=64)

X_sample = np.stack(df_silhouette_sample['embeddings'].values)
labels_sample = df_silhouette_sample['title_cluster'].values

Wow, this is useless.

In [22]:
import pickle
pkl_fname = "title-kmeans.pkl"
with open(pkl_fname, 'wb') as file:
    pickle.dump(kmeans, file)

In [ ]:
"""
# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
"""

## Sheer utter boredom

let's predict the HN post score in the dumbest way possible.

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [72]:
bins = [0, 32, 64, 128, 256, 512, 1024, 2048, float('inf')]
labels = ['0-32', '32-64', '64-128', '128-256', '256-512', '512-1024', '1024-2048', '2048+']

df['score_range'] = pd.cut(df['score'], bins=bins, labels=labels, right=False)
df['score_label'] = df['score_range'].astype('category').cat.codes


In [73]:
df["score_label"].value_counts()

score_label
1    126513
2    112603
0     86444
3     79324
4     34975
5      9261
6      1455
7       158
Name: count, dtype: int64

In [128]:
# Separate the data by 'score_label'
grouped = df.groupby('score_label')

# Determine the number of samples to draw from each group to balance them
# Here, we aim to match the 80% size of the smallest group (bucket 7)
target_size = int(0.8 * grouped.size().min())

# Sample from each group
sampled_dfs = [group.sample(target_size, replace=True) for _, group in grouped]

# Combine the sampled data to create a balanced training set
balanced_df = pd.concat(sampled_dfs, ignore_index=True)


embeddings = np.array(balanced_df["embeddings"].tolist())
labels = balanced_df["score_label"].values

In [141]:
import torch

X = torch.tensor(embeddings, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)

In [142]:
from torch.utils.data import DataLoader, TensorDataset
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DataLoader
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)

In [132]:
import torch.nn as nn

# Define the model
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.fc(x)

In [150]:
import torch.optim as optim
# Initialize the model, loss, and optimizer
model = SimpleClassifier(X_train.shape[1], len(bins) - 1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [143]:
y_train.shape

torch.Size([806])

In [151]:
# Training loop
for epoch in range(10):  # 10 epochs as an example
    for batch_X, batch_y in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch}, Loss: {loss.item()}")



Epoch 0, Loss: 2.093410015106201
Epoch 1, Loss: 1.9829645156860352
Epoch 2, Loss: 2.014063835144043
Epoch 3, Loss: 1.9200794696807861
Epoch 4, Loss: 1.9282312393188477
Epoch 5, Loss: 1.9511048793792725
Epoch 6, Loss: 2.049727439880371
Epoch 7, Loss: 1.81847083568573
Epoch 8, Loss: 2.116856336593628
Epoch 9, Loss: 1.804591178894043


In [91]:
len(bins)

9

In [152]:
# DataLoader for test set
test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Evaluate the model
correct = 0
total = 0

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        # Forward pass
        outputs = model(batch_X)
        
        # Get the index of the highest score in the output tensor
        _, predicted = torch.max(outputs.data, 1)
        
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 16.83%


In [153]:
# Assuming 'model' is your trained PyTorch model and 'df' is your DataFrame

# Convert embeddings to PyTorch tensor
embeddings = torch.tensor(df['embeddings'].tolist(), dtype=torch.float32)

# DataLoader
data_loader = DataLoader(embeddings, batch_size=2048, shuffle=False)

# Store predictions
predictions = []

# Evaluate
with torch.no_grad():
    for batch_X in data_loader:
        # Forward pass
        outputs = model(batch_X)
        
        # Get the index of the highest score in the output tensor
        _, predicted = torch.max(outputs.data, 1)
        
        predictions.extend(predicted.tolist())

# Add predictions to DataFrame
df['predicted_label'] = predictions


In [154]:
# Create a new boolean column that is True if the predicted and actual labels match, otherwise False
df['is_correct'] = df['score_label'] == df['predicted_label']

# Group by the cluster ID and calculate both the mean and count of 'is_correct' for each group
cluster_stats = df.groupby('title_cluster')['is_correct'].agg(['mean', 'count'])

# Rename columns for better understanding
cluster_stats.columns = ['mean_accuracy', 'count']

# Sort the DataFrame if needed
cluster_stats = cluster_stats.sort_values(by=['count', 'mean_accuracy'], ascending=[False, False])

In [118]:
cluster_stats.to_csv("datasets/cluster_stats_128")

In [155]:
# Group by the actual score label bins and calculate the mean accuracy for each group
score_bin_stats = df.groupby('score_label')['is_correct'].agg(['mean', 'count'])

# Sort the DataFrame based on mean accuracy or count as you prefer
score_bin_stats = score_bin_stats.sort_values(by=['mean', 'count'], ascending=[False, False])

# Rename columns for better understanding
score_bin_stats.columns = ['mean_accuracy', 'count']
score_bin_stats

,mean_accuracy,count
score_label,,
6,0.426117,1455
7,0.316456,158
1,0.239912,126513
0,0.148350,86444
3,0.132595,79324
5,0.092107,9261
4,0.083145,34975
2,0.079634,112603
